#                                           Transfer Learning using PyTorch

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [ ]:
df = pd.read_csv('/content/fmnist_small.csv')
df.shape

(6000, 785)

- Train_test_split

In [ ]:
from numpy.random.mtrand import random
x = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=22)

In [ ]:
x_train = x_train/255.0
x_test = x_test/255.0

Transformations

In [ ]:
from torchvision import transforms

In [ ]:
custom_transform = transforms.Compose([
    transforms.Resize((224,224)),
    # transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])

In [ ]:
from PIL import Image
import numpy as np

class customdset(Dataset):

  def __init__(self, features, labels, transform):
    super().__init__()
    self.features = features
    self.labels = labels
    self.transform = transform

  def __len__(self):
    return len(self.features)


  def __getitem__(self, index):
    # resize to (28, 28)
    image = self.features[index].reshape(28,28)
    # change data type to np.uint8
    image = image.astype(np.uint8)
    # change black & white to color
    image = np.stack([image]*3, axis = -1) #(3, 28, 28) # PIL want to img format as (width, height, channels) so we should use axis =-1
    # convert array to PIL image
    image = Image.fromarray(image)
    # apply transformation
    image = self.transform(image)
    # return index
    return image, torch.tensor(self.labels[index], dtype = torch.long)

Dataset & Loader

In [ ]:
test_dataset = customdset(x_test, y_test, transform = custom_transform)
train_dataset = customdset(x_train, y_train, transform = custom_transform)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size = 64, shuffle = False, pin_memory=True)

- import pre-trained model

In [ ]:
import torchvision.models as models

vgg16 = models.vgg16(pretrained=True)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# freeze features training
for param in vgg16.features.parameters():
  param.requires_grad = False

In [ ]:
# replace classifier with our new classifier
vgg16.classifier = nn.Sequential(
    nn.Linear(25088, 1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 10)
    )

In [ ]:
vgg16.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=1024, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=1024, out_features=512, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=512, out_features=10, bias=True)
)

In [ ]:
vgg16 = vgg16.to(device)

In [ ]:
learning_rate = 0.001
epochs = 15

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.classifier.parameters(), lr = learning_rate)

Training LOOP

In [ ]:
for epoch in range(epochs):
  total_epoch_loss = 0

  for batch_features, batch_labels in train_loader:
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)# move data to GPU
    outputs = vgg16(batch_features) # forward pass
    loss = criterion(outputs, batch_labels) # loss calculation
    optimizer.zero_grad() # clear gradients
    loss.backward() # backward pass
    optimizer.step() # update weights
    total_epoch_loss += loss.item()
  avg_loss = total_epoch_loss/len(train_loader)
  print(f"epoch: {epoch+1}, loss: {avg_loss}")


epoch: 1, loss: 2.328344462712606
epoch: 2, loss: 2.294149227142334
epoch: 3, loss: 2.272504065831502
epoch: 4, loss: 2.2487198225657146
epoch: 5, loss: 2.215827252070109
epoch: 6, loss: 2.2004471397399903
epoch: 7, loss: 2.1750100485483803
epoch: 8, loss: 2.1748464504877725
epoch: 9, loss: 2.1885434786478677
epoch: 10, loss: 2.1547291310628256
epoch: 11, loss: 2.1548508739471437
epoch: 12, loss: 2.1634185949961346
epoch: 13, loss: 2.1162214263280235
epoch: 14, loss: 2.1332523727416994
epoch: 15, loss: 2.116058071454366


In [ ]:
vgg16.eval()


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
total = 0
correct = 0

with torch.no_grad():
  for batch_features, batch_labels in test_loader:
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = vgg16(batch_features)

    _, predicted = torch.max(outputs.data, 1)

    total = total + batch_labels.size(0)

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.20666666666666667
